In [1]:
import requests, time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + '1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
trs = soup.select('form > table > tbody > tr')
len(trs)

100

In [4]:
trs = soup.select('.aos-init')
len(trs)

0

In [5]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + '1'
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [7]:
len(driver.page_source), len(res.text)

(296457, 292070)

In [8]:
tr = trs[0]
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [9]:
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [10]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [11]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'8730만'

In [12]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view, video

('305억3350만', '507개')

In [13]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [14]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    lines.append({
        '순위':rank, '카테고리':category, '채널명':channel,
        '구독자수':subscriber, '조회수':view, '비디오':video
    })

In [15]:
df1 = pd.DataFrame(lines)
df1.tail(7)

,순위,카테고리,채널명,구독자수,조회수,비디오
93,94,음식/요리/레시피,떵개떵,4890000,2981590000,4910
94,96,키즈/어린이,ToyMart TV,4860000,1993230000,687
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450
99,100,미분류,매일맛나 delicious day,4740000,482360000,325


In [16]:
lines = []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
for page in range(1, 11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({
            '순위':rank, '카테고리':category, '채널명':channel,
            '구독자수':subscriber, '조회수':view, '비디오':video
        })

driver.close()

In [17]:
df = pd.DataFrame(lines)
df.shape

(1000, 6)

In [18]:
df.to_csv('data/20230425_YouTube랭킹.csv', index=False)